In [50]:
import sys
sys.path.append('/Users/celia/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celia/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import os
import behavior_database
import shutil
import ipywidgets as widgets
from IPython.display import display

In [51]:
record_df = pd.read_csv('/Users/celia/GitHub/mouse_bandit/celia_session_record.csv',index_col=0)
celia_session_record = True

### First time we ran this, had to add 'shay' to the old data base and add a new column for Markov
    if need to troubleshoot or add trials from older data, use creating_behavior_database notebook

In [52]:
print(record_df.shape)
record_df.tail(3)

(701, 18)


Owner     Session ID Mouse ID        Date  Phase  Markov  \
698  celia  12202017_Dill     Dill  2017-12-20    2.0       1   
699  celia  12212017_Dill     Dill  2017-12-21    2.0       1   
700  celia  12222017_Dill     Dill  2017-12-22    2.0       1   

     Left Reward Prob  Right Reward Prob  Block Range Min  Block Range Max  \
698               0.2                0.8              NaN              NaN   
699               0.8                0.2              NaN              NaN   
700               0.2                0.8              NaN              NaN   

     No. Trials  No. Blocks  No. Rewards  p(high Port)  \
698       775.0        15.0        494.0          0.74   
699       831.0        19.0        551.0          0.78   
700       863.0        19.0        548.0          0.71   

     Decision Window Duration  Min Inter-trial-interval  \
698                       2.0                       1.0   
699                       2.0                       1.0   
700                       2.0                       1.0   

     Left Solenoid Duration  Right Solenoid Duration  
698                    40.0                     40.0  
699                    40.0                     40.0  
700                    35.0                     35.0

## Add all days (multi mice) to celia_session_record

In [53]:
if celia_session_record == True:
    record_df = record_df.drop(record_df.index[:]) # if want to reset and start with (for new session record)
    
    MOUSE = ['Baby','Mustard','Raspberry','Sky','Wine', 'Dill']
    ROOT = '/Users/celia/Dropbox (HMS)/Celia_BehaviorFiles/'
    root_mouse={}
    for mouse in MOUSE: # make list of all mouse directories
        temp_dict = {mouse:os.path.join(ROOT,mouse)}
        root_mouse.update(temp_dict)

    date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
    date_folders = date_folders[1:]
    count=0
    for mouse in MOUSE:
        for date in date_folders:
            curr_dir = os.path.join(root_mouse[mouse], date)
            record_df = behavior_database.add_session(curr_dir, record_df, 'celia')
            count+=1
    record_df.to_csv(path_or_buf='/Users/celia/GitHub/mouse_bandit/celia_session_record.csv')

In [54]:
count - 60 # estimate for number of sessions

720

In [55]:
record_df.shape # true number of sessions in record

(701, 18)

## Moving files around
    Use MovingFilesAround notebook for moving data outside of new structure

In [56]:
# files from Celia

root_dir = '/Users/celia/Dropbox (HMS)/Celia_BehaviorFiles/'
MOUSE = ['Baby','Mustard','Raspberry','Sky','Wine']
root_mouse={}
for mouse in MOUSE: # make list of all mouse directories
    temp_dict = {mouse:os.path.join(root_dir,mouse)}
    root_mouse.update(temp_dict)
date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
date_folders = date_folders[1:]

dist_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'

for mouse in MOUSE:
    for date in date_folders:
        for file in os.listdir(os.path.join(root_mouse[mouse], date)):

            if not file[0] == '.':
                file_name = os.path.join(root_mouse[mouse],date,file)
                if 'trials.csv' in file:
                    shutil.copy(file_name, dist_dir)
                    #print(file_name)
                else:
                    pass
                    #print(date_folders)
                    #print(root_mouse[mouse])
                    #raise ValueError('In a hidden folder')
                    
MOUSE = ['Dill']
root_mouse={}
for mouse in MOUSE: # make list of all mouse directories
    temp_dict = {mouse:os.path.join(root_dir,mouse)}
    root_mouse.update(temp_dict)
date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
date_folders = date_folders[1:]

dist_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'

for mouse in MOUSE:
    for date in date_folders:
        for file in os.listdir(os.path.join(root_mouse[mouse], date)):

            if not file[0] == '.':
                file_name = os.path.join(root_mouse[mouse],date,file)
                if 'trials.csv' in file:
                    shutil.copy(file_name, dist_dir)
                    #print(file_name)
                else:
                    pass
                    #print(date_folders)
                    #print(root_mouse[mouse])
                    #raise ValueError('In a hidden folder')
        

## Make feature matrix for each condition type

In [57]:
db = pd.read_csv('/Users/celia/GitHub/mouse_bandit/celia_session_record.csv',index_col=0)
db.shape # just to double check size updated

(701, 18)

#### all markov sessions with >0.7 performance

In [58]:
r = db[(db['Markov']==1)]
r = r[r['p(high Port)'] > 0.7].copy()
r = r[r['No. Blocks']>0.0].copy()
session_names = r['Session ID'].values

In [59]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame', 'Block ID']

root_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'
save_dir = '/Users/celia/GitHub/mouse_bandit/data/processed_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [60]:
progress = widgets.IntProgress(min=0, max=len(trial_df), step=1)
display(progress)
for i,df in enumerate(trial_df):
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    progress.value+=1

In [61]:
master_matrix.to_csv(os.path.join(save_dir,'markov_master.csv'),index=True)

##### 70-30 sessions

In [70]:
r = db[(db['Markov']==1)]
r = r[((r['Left Reward Prob'] == 0.7) |  (db['Left Reward Prob'] == 0.3))].copy()
r = r[r['p(high Port)'] > 0.7].copy()
r = r[r['No. Blocks']>0.0].copy()
session_names = r['Session ID'].values

In [71]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame', 'Block ID']

root_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'
save_dir = '/Users/celia/GitHub/mouse_bandit/data/processed_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [72]:
progress = widgets.IntProgress(min=0, max=len(trial_df), step=1)
display(progress)
for i,df in enumerate(trial_df):
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    progress.value+=1

In [73]:
master_matrix.to_csv(os.path.join(save_dir,'markov_7030.csv'),index=True)